<a href="https://colab.research.google.com/github/Maheshmrn/book-sphere/blob/main/FinalBookSphere.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install streamlit pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 4.5 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [124]:
with open("app.py", "w") as f:
    f.write("""
import streamlit as st
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
rating = pd.read_csv('/content/drive/MyDrive/Ratings.csv.zip')
books = pd.read_csv('/content/drive/MyDrive/Books.csv.zip')
rating_with_name = rating.merge(books,on='ISBN')
#popularity based code
num_rating = rating_with_name.groupby('Book-Title').count()['Book-Rating'].reset_index()
num_rating.rename(columns={'Book-Rating' : 'num-rating'}, inplace = True)
avg_rating = rating_with_name.groupby('Book-Title')['Book-Rating'].mean().reset_index()
avg_rating.rename(columns={'Book-Rating' : 'avg-rating'},inplace = True)
popular_books = num_rating.merge(avg_rating, on = 'Book-Title')
popular_books = popular_books[popular_books['num-rating']>=250].sort_values('avg-rating',ascending= False).head(50)
popular_books = popular_books.merge(books, on = 'Book-Title').drop_duplicates('Book-Title')[['Book-Title','Book-Author','Image-URL-M','num-rating','avg-rating']]
#collaborative based code
x = rating_with_name.groupby('User-ID').count()['Book-Rating'] >= 200
most_rating_users = x[x].index
filtered_rating = rating_with_name[rating_with_name['User-ID'].isin(most_rating_users)]
y = filtered_rating.groupby('Book-Title').count()['Book-Rating'] >= 50
famous_books = y[y].index
final_rating = filtered_rating[filtered_rating['Book-Title'].isin(famous_books)]
pt = final_rating.pivot_table(index='Book-Title', columns='User-ID', values='Book-Rating')
pt.fillna(0, inplace=True)
similar_scores = cosine_similarity(pt)
# collabative-based function
def bookSphere(book_name):
    index = np.where(pt.index == book_name)[0][0]
    similar_books = sorted(list(enumerate(similar_scores[index])), key=lambda x:x[1], reverse=True)[1:12]
    results = []
    for i in similar_books:
        books1 = books[books['Book-Title'] == pt.index[i[0]]]
        book_info = []
        book_info.extend(list(books1.drop_duplicates('Book-Title')['Book-Title'].values))
        book_info.extend(list(books1.drop_duplicates('Book-Title')['Book-Author'].values))
        book_info.extend(list(books1.drop_duplicates('Book-Title')['Image-URL-M'].values))
        results.append(book_info)
    return results
# Popularity-based function
def popular_books_list():
    results = []
    for title in popular_books['Book-Title']:
        books1 = books[books['Book-Title'] == title].drop_duplicates('Book-Title')
        book_info = []
        book_info.extend(list(books1['Book-Title'].values))
        book_info.extend(list(books1['Book-Author'].values))
        book_info.extend(list(books1['Image-URL-M'].values))
        results.append(book_info)
    return results
# Streamlit UI
st.set_page_config(page_title="📚 BookSphere", layout="wide")
st.title("📚 BookSphere")
st.subheader("Smart Book Recommendations with AI")
book_list = pt.index.sort_values().tolist()
book_list.insert(0, "")
col1, col2 = st.columns([1, 9])
with col1:
    if st.button("🏠 Home"):
      st.session_state.selected_book = ""
      st.rerun()
if 'history' not in st.session_state:
    st.session_state.history = []
if 'selected_book' not in st.session_state:
    st.session_state.selected_book = ""
typed_book = st.text_input("Start typing...", "")
matching_books = [title for title in pt.index if typed_book.lower() in title.lower()]
selected = None
if typed_book.strip() and matching_books:
    st.markdown("📚 Suggestions:")
    for mbook in matching_books[:5]:
        if st.button(mbook):
            st.session_state.selected_book = mbook
            selected = mbook
book_to_search = st.session_state.selected_book or typed_book
show_recommendation = st.button("Get Recommendations")
if st.button("🗑️ Clear Search History"):
    st.session_state.history = []
if show_recommendation and book_to_search.strip() != "":
    if book_to_search not in st.session_state.history:
        st.session_state.history.append(book_to_search)
    st.subheader(f"📘 Books similar to '{book_to_search}'")
    try:
        recs = bookSphere(book_to_search)
        cols = st.columns(3)
        for idx, book in enumerate(recs):
            title, author, img = book
            with cols[idx % 3]:
                st.image(img, width=150)
                st.markdown(f"{title}")
                st.markdown(f"{author}")
                search_url = f"https://www.goodreads.com/search?q={title.replace(' ', '+')}"
                st.markdown(f"[🔗 More Info]({search_url})", unsafe_allow_html=True)
    except:
        st.error("Book not found or not enough data.")
elif not book_to_search:
    st.subheader("🔥 Most Popular Books Right Now")
    pop_recs = popular_books_list()
    cols = st.columns(4)
    for idx, book in enumerate(pop_recs):
        title, author, img = book
        with cols[idx % 4]:
            st.image(img, width=150)
            st.markdown(f"{title}")
            st.markdown(f"{author}")
            search_url = f"https://www.goodreads.com/search?q={title.replace(' ', '+')}"
            st.markdown(f"[🔗 More Info]({search_url})", unsafe_allow_html=True)
if st.session_state.history:
    st.markdown("🕘 Recently Searched:")
    for recent in reversed(st.session_state.history[-5:]):
        st.markdown(f"🔹{recent}")
""")

In [125]:
!pkill streamlit
!killall ngrok

In [126]:
!ngrok authtoken 2vaf6AFGfMb9KmgkK1QjDn1mukl_5qP6QVzJtWiZeX9nar9G3
from pyngrok import ngrok
public_url = ngrok.connect("http://localhost:8501")
print("🚀 Your live app: ", public_url)
!streamlit run app.py &>/dev/null &

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
🚀 Your live app:  NgrokTunnel: "https://7eec-104-199-178-216.ngrok-free.app" -> "http://localhost:8501"
